In [1]:
from data_cleaning import loading_data
from baseline_model import baseline
from linear_model import linear_regression
from extra_trees_model import extra_trees_regressor

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn import tree

In [2]:
data = loading_data("train", "store")

/Users/akasper/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
#importing libraries
import pandas as pd
import numpy as np
from math import sqrt

#dropping columns and rows with null values
def delete_nulls(data):
    data = data.drop(columns=["CompetitionOpenSinceMonth", "CompetitionOpenSinceYear",
                              "Promo2SinceWeek", "Promo2SinceYear", "PromoInterval", 
                              "Date", "StateHoliday", "SchoolHoliday", 
                              "Assortment", "StoreType"])
    data = data[~(data.loc[:, "DayOfWeek"].isnull()) &
                ~(data.loc[:, "Sales"].isnull()) &
                ~(data.loc[:, "Customers"].isnull()) &
                ~(data.loc[:, "Open"].isnull()) &
                ~(data.loc[:, "Promo"].isnull()) &
                ~(data.loc[:, "CompetitionDistance"].isnull()) &
                ~(data.loc[:, "Sales"] == 0.0)]
    return data

# splitting features and target back apart
def split(data):
    x_train = data.copy(deep=True).drop(columns=["Sales"])
    y_train = data.loc[:, "Sales"]
    return x_train, y_train

# using the mean of the entire training set as a first prediction
def lazy_estimator(target):
    lazy_estimator_predictions = pd.DataFrame(target.copy())
    lazy_estimator_predictions.loc[:, 'lazy_predicted_price'] = target.mean()
    predict = lazy_estimator_predictions.loc[:, 'lazy_predicted_price']
    return predict

# defining evaluation metric
def compute_rmspe(actual, prediction):
    rmspe = np.sqrt(np.mean(np.square(((actual - prediction) / actual)), axis=0)) 
    return rmspe

# our baseline model using the functions defined above
def baseline(data):
    # deleting rows with null values and getting rid of some columns
    cleaned_data = delete_nulls(data)
    
    # splitting the data into features and target
    X, y = split(cleaned_data)
    
    # calculating the prediction which is simply the mean here
    predict = lazy_estimator(y)
    
    # computing the RMSPE of the difference between the prediction and the target
    rmspe = compute_rmspe(y, predict)

    print("the RMSPE of the baseline model (mean) is {}".format(rmspe.round(4)))

In [4]:
cleaned_data = delete_nulls(data)

In [5]:
cleaned_data

,Store,DayOfWeek,Sales,Customers,Open,Promo,CompetitionDistance,Promo2
27,353,2.0,3139.0,820.0,1.0,0.0,900.0,1
115,335,2.0,2401.0,482.0,1.0,0.0,90.0,1
145,512,2.0,2646.0,625.0,1.0,0.0,590.0,1
160,494,2.0,3113.0,527.0,1.0,0.0,1260.0,0
194,530,2.0,2907.0,532.0,1.0,0.0,18160.0,0
...,...,...,...,...,...,...,...,...
618467,743,4.0,4527.0,473.0,1.0,1.0,6710.0,1
618468,752,4.0,7259.0,576.0,1.0,1.0,970.0,1
618470,754,4.0,10564.0,1162.0,1.0,1.0,380.0,1
618471,755,4.0,12302.0,1020.0,1.0,1.0,13130.0,0


In [6]:
data

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,2013-01-01,1115,2.0,0.0,0.0,0.0,0.0,a,1.0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1,2013-01-01,379,2.0,0.0,0.0,0.0,0.0,a,1.0,d,a,6630.0,NaN,NaN,0,NaN,NaN,NaN
2,2013-01-01,378,2.0,0.0,0.0,0.0,0.0,a,1.0,a,c,2140.0,8.0,2012.0,0,NaN,NaN,NaN
3,2013-01-01,377,2.0,0.0,0.0,0.0,0.0,a,1.0,a,c,100.0,6.0,2010.0,1,18.0,2010.0,"Feb,May,Aug,Nov"
4,2013-01-01,376,2.0,0.0,0.0,0.0,0.0,a,1.0,a,a,160.0,8.0,2012.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618468,2014-07-31,752,4.0,7259.0,576.0,1.0,1.0,0,1.0,a,a,970.0,3.0,2013.0,1,31.0,2013.0,"Feb,May,Aug,Nov"
618469,2014-07-31,753,NaN,9938.0,796.0,1.0,1.0,0,1.0,d,c,540.0,11.0,2012.0,1,35.0,2010.0,"Mar,Jun,Sept,Dec"
618470,2014-07-31,754,4.0,10564.0,1162.0,1.0,1.0,0,NaN,c,c,380.0,5.0,2008.0,1,10.0,2014.0,"Mar,Jun,Sept,Dec"
618471,2014-07-31,755,4.0,12302.0,1020.0,1.0,1.0,0,1.0,d,c,13130.0,12.0,2003.0,0,NaN,NaN,NaN


In [7]:
data['Promo'].sum()

222150.0

In [8]:
samp = data.sample(frac=0.2)

In [9]:
samp

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
138828,2013-05-09,41,4.0,0.0,0.0,0.0,0.0,a,0.0,d,c,1180.0,NaN,NaN,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
405200,2014-01-10,1007,5.0,8663.0,898.0,1.0,1.0,0,0.0,c,c,4180.0,9.0,2012.0,0,NaN,NaN,NaN
125089,2013-04-26,1104,5.0,5759.0,435.0,1.0,1.0,0,0.0,d,a,260.0,2.0,2012.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
486996,2014-03-27,210,4.0,2292.0,278.0,1.0,0.0,0,0.0,d,a,970.0,11.0,1999.0,1,5.0,2013.0,"Feb,May,Aug,Nov"
148586,2013-05-18,230,6.0,6876.0,909.0,1.0,0.0,0,0.0,d,c,6680.0,9.0,2013.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575822,2014-06-17,952,2.0,7158.0,510.0,1.0,1.0,0,1.0,d,c,6300.0,10.0,2013.0,0,NaN,NaN,NaN
271074,2013-09-08,1056,7.0,0.0,0.0,0.0,0.0,0,0.0,d,c,5350.0,NaN,NaN,1,40.0,2012.0,"Jan,Apr,Jul,Oct"
417398,2014-01-21,704,2.0,10783.0,1094.0,1.0,1.0,0,0.0,d,c,1910.0,4.0,2009.0,0,NaN,NaN,NaN
467553,2014-03-09,252,7.0,0.0,0.0,0.0,0.0,0,NaN,d,c,22330.0,NaN,NaN,1,5.0,2010.0,"Feb,May,Aug,Nov"


In [12]:

dtc = tree.DecisionTreeRegressor(max_depth=5, random_state=42, criterion='mse')

dtc.fit(X, y)
predict = dtc.predict(x_test)
dtc.score(x_test, y_test)

NameError: name 'X' is not defined